In [1]:
!pip install -q sklearn

In [2]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [96]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing

In [4]:
from google.colab import files
uploaded = files.upload()

Saving House_Rent_Dataset.csv to House_Rent_Dataset.csv


In [65]:
import io
df2 = pd.read_csv(io.BytesIO(uploaded['House_Rent_Dataset.csv']))
# Dataset is now stored in a Pandas Dataframe

In [66]:
df2.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner


In [67]:
df2["Floor_Number"]=df2["Floor"].apply(lambda x:str(x).split()[0])

In [68]:
df2["Total_Number_of_Floor"]=df2["Floor"].apply(lambda x:str(x).split()[-1])

In [69]:
df2.isnull().sum()

Posted On                0
BHK                      0
Rent                     0
Size                     0
Floor                    0
Area Type                0
Area Locality            0
City                     0
Furnishing Status        0
Tenant Preferred         0
Bathroom                 0
Point of Contact         0
Floor_Number             0
Total_Number_of_Floor    0
dtype: int64

In [71]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Posted On              4746 non-null   object
 1   BHK                    4746 non-null   int64 
 2   Rent                   4746 non-null   int64 
 3   Size                   4746 non-null   int64 
 4   Floor                  4746 non-null   object
 5   Area Type              4746 non-null   object
 6   Area Locality          4746 non-null   object
 7   City                   4746 non-null   object
 8   Furnishing Status      4746 non-null   object
 9   Tenant Preferred       4746 non-null   object
 10  Bathroom               4746 non-null   int64 
 11  Point of Contact       4746 non-null   object
 12  Floor_Number           4746 non-null   object
 13  Total_Number_of_Floor  4746 non-null   object
dtypes: int64(4), object(10)
memory usage: 519.2+ KB


In [73]:
df2.drop(["Posted On","Floor"], axis="columns",inplace=True)

In [74]:
df2.head()

,BHK,Rent,Size,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact,Floor_Number,Total_Number_of_Floor
0,2,10000,1100,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner,Ground,2
1,2,20000,800,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner,1,3
2,2,17000,1000,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner,1,3
3,2,10000,800,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner,1,2
4,2,7500,850,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner,1,2


In [75]:
x = df2["Floor_Number"]=="Ground"

In [76]:
df2[x]

,BHK,Rent,Size,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact,Floor_Number,Total_Number_of_Floor
0,2,10000,1100,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner,Ground,2
5,2,7000,600,Super Area,Thakurpukur,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner,Ground,1
6,2,10000,700,Super Area,Malancha,Kolkata,Unfurnished,Bachelors,2,Contact Agent,Ground,4
14,3,8500,900,Carpet Area,Paschim Barisha,Kolkata,Unfurnished,Bachelors,2,Contact Owner,Ground,2
17,2,10000,800,Super Area,Behala,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner,Ground,2
...,...,...,...,...,...,...,...,...,...,...,...,...
4721,1,5500,40,Super Area,Jagadgiri Gutta,Hyderabad,Unfurnished,Bachelors/Family,1,Contact Owner,Ground,2
4722,1,8000,850,Super Area,Hyderguda,Hyderabad,Unfurnished,Bachelors/Family,1,Contact Owner,Ground,3
4729,2,13000,1500,Carpet Area,"Saket, Secunderabad",Hyderabad,Unfurnished,Family,2,Contact Owner,Ground,1
4732,2,7000,900,Super Area,Vinayaka Nagar,Hyderabad,Unfurnished,Bachelors/Family,1,Contact Owner,Ground,2


In [77]:
def fun(c):
  return c.replace("Ground","0")

In [78]:
clm = ["Floor_Number"]
for c in clm:
  df2[c]=df2[c].apply(fun)

In [79]:
df2["Floor_Number"].value_counts()

1        1161
2         945
0         927
3         512
4         272
5         164
6          93
7          74
10         67
8          66
9          65
12         47
11         43
15         41
14         34
18         26
Upper      23
17         22
16         21
19         16
13         15
20         12
25         12
Lower      11
23          9
24          6
21          6
30          5
34          4
28          4
35          3
65          3
22          3
26          3
32          3
60          3
27          3
40          2
53          2
45          2
36          2
48          2
29          1
49          1
50          1
39          1
46          1
41          1
44          1
33          1
37          1
47          1
43          1
76          1
Name: Floor_Number, dtype: int64

In [80]:
#replace lower with 1
def fun2(v):
  return v.replace("Lower","1")

In [81]:
for v in clm:
  df2[v]=df2[v].apply(fun2)

In [82]:
#replace upper with number of floors
def fun3(b):
  return b

In [83]:
df2["Floor_Number"]=df2["Total_Number_of_Floor"].apply(fun3)

In [84]:
df2.head(100)

,BHK,Rent,Size,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact,Floor_Number,Total_Number_of_Floor
0,2,10000,1100,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner,2,2
1,2,20000,800,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner,3,3
2,2,17000,1000,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner,3,3
3,2,10000,800,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner,2,2
4,2,7500,850,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,5000,300,Carpet Area,Dum Dum Metro,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner,2,2
96,2,12000,1200,Super Area,Boral,Kolkata,Furnished,Bachelors/Family,2,Contact Owner,3,3
97,2,10000,500,Carpet Area,"Rabindra Pally, Keshtopur",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner,4,4
98,4,18000,1200,Super Area,Bagbazar,Kolkata,Unfurnished,Family,2,Contact Owner,2,2


In [85]:
df2.tail(100)

,BHK,Rent,Size,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact,Floor_Number,Total_Number_of_Floor
4646,2,14500,1100,Super Area,Toli Chowki,Hyderabad,Semi-Furnished,Bachelors/Family,2,Contact Owner,5,5
4647,1,8000,450,Super Area,Puppalaguda,Hyderabad,Unfurnished,Bachelors/Family,1,Contact Owner,3,3
4648,6,80000,4500,Super Area,"Kakateeya Nagar, Secunderabad",Hyderabad,Semi-Furnished,Bachelors/Family,6,Contact Owner,2,2
4649,2,15000,1300,Carpet Area,5-20 Adharshnagar,Hyderabad,Semi-Furnished,Bachelors/Family,2,Contact Owner,4,4
4650,1,15500,1000,Carpet Area,"Manikonda, Outer Ring Road",Hyderabad,Semi-Furnished,Bachelors/Family,1,Contact Owner,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...
4741,2,15000,1000,Carpet Area,Bandam Kommu,Hyderabad,Semi-Furnished,Bachelors/Family,2,Contact Owner,5,5
4742,3,29000,2000,Super Area,"Manikonda, Hyderabad",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Owner,4,4
4743,3,35000,1750,Carpet Area,"Himayath Nagar, NH 7",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Agent,5,5
4744,3,45000,1500,Carpet Area,Gachibowli,Hyderabad,Semi-Furnished,Family,2,Contact Agent,34,34


In [94]:
df2.drop(["Bathroom"], axis="columns", inplace=True)

In [107]:
#convert to numeric
clms=["Area Type", "Area Locality", "City", "Furnishing Status", "Tenant Preferred", "Point of Contact","Total_Number_of_Floor","Floor_Number"]

In [108]:
le=preprocessing.LabelEncoder()

In [109]:
for n in clms:
    df2[n]=le.fit_transform(df2[n])

In [110]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   BHK                    4746 non-null   int64
 1   Rent                   4746 non-null   int64
 2   Size                   4746 non-null   int64
 3   Area Type              4746 non-null   int64
 4   Area Locality          4746 non-null   int64
 5   City                   4746 non-null   int64
 6   Furnishing Status      4746 non-null   int64
 7   Tenant Preferred       4746 non-null   int64
 8   Point of Contact       4746 non-null   int64
 9   Floor_Number           4746 non-null   int64
 10  Total_Number_of_Floor  4746 non-null   int64
dtypes: int64(11)
memory usage: 408.0 KB


In [111]:
#split to training and testing
p=df2.drop("Rent", axis="columns")
q=df2["Rent"]

In [170]:
ptrain,ptest,qtrain,qtest=train_test_split(p,q,train_size=0.85, random_state=2)

In [171]:
lr = LinearRegression()
pf = PolynomialFeatures(degree=2)
Xp = pf.fit_transform(p)

In [172]:
lr.fit(Xp,q)

LinearRegression()

In [173]:
from sklearn.feature_selection import SelectFromModel

In [174]:
sm = SelectFromModel(lr,threshold=10)
Xt = sm.fit_transform(Xp,q)
sm_estimator = sm.estimator_.score(Xp,q)
print(sm_estimator)

0.398648668416248
